<a href="https://colab.research.google.com/github/mentaldropouts/Quote-Generation-ML/blob/main/CS460Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gdown
from google.colab import drive
url = "https://drive.google.com/uc?id=1lyo7edhxQfimeXq45kjsp8y9XuXs9gej"
output = "quotes.csv"
gdown.download(url, output, quiet = False)
drive.mount('/content/drive')

Downloading...
From: https://drive.google.com/uc?id=1lyo7edhxQfimeXq45kjsp8y9XuXs9gej
To: /content/quotes.csv
100%|██████████| 165M/165M [00:01<00:00, 163MB/s]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:

#Reading in CSV file and dealing with Column issues

quotesDF = pd.read_csv(output, header = None, nrows = 5000)
UnecessaryCols = [col for col in quotesDF.columns if col > 2]
#print(UnecessaryCols)
quotesDF.drop(columns = UnecessaryCols, inplace = True)
quotesDF.columns = ["Quote", "Author","Tags"]
quotesDF.head()

,Quote,Author,Tags
0,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,"attributed-no-source, best, life, love, mistak..."
1,You've gotta dance like there's nobody watchin...,William W. Purkey,"dance, heaven, hurt, inspirational, life, love..."
2,You know you're in love when you can't fall as...,Dr. Seuss,"attributed-no-source, dreams, love, reality, s..."
3,A friend is someone who knows all about you an...,Elbert Hubbard,"friend, friendship, knowledge, love"
4,Darkness cannot drive out darkness: only light...,"Martin Luther King Jr., A Testament of Hope: T...","darkness, drive-out, hate, inspirational, ligh..."


In [4]:
import sklearn.model_selection
import re

In [5]:
Train_ValidDF, testDF = sklearn.model_selection.train_test_split(quotesDF, test_size = .2 )
trainDF, validDF = sklearn.model_selection.train_test_split(Train_ValidDF, test_size = .2)

In [6]:
def editQuotes(aDF):
  newQuotes = []
  for quote in aDF['Quote'].tolist():
    quote = str(quote).strip()
    #DIFFERENTLY DONE
    quote = " ".join(quote.split())
    bufferize = [" ", quote, " "]
    newQuotes.append(' '.join(bufferize))
  return newQuotes

trainDF = editQuotes(trainDF)
testDF = editQuotes(testDF)
validDF = editQuotes(validDF)

In [7]:
def turnToText(contents, filename):
  with open(filename, 'w', encoding='utf-8') as fhead:
    for content in contents:
      fhead.write(content+'\n')

In [8]:
turnToText(trainDF, 'train.txt')
turnToText(testDF, 'test.txt')
turnToText(validDF, 'valid.txt')

In [9]:
from google.colab import files

In [10]:
!pip install transformers[torch]
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.5 MB/s eta 0:00:00


In [11]:
!python3 '/content/drive/MyDrive/model.py' \
--output_dir='/content/drive/MyDrive/model' \
--overwrite_output_dir \
--model_name_or_path='/content/drive/MyDrive/model/checkpoint' \
--tokenizer_name=gpt2 \
--do_train \
--train_data_file='/content/train.txt' \
--do_eval \
--eval_data_file='/content/valid.txt' \
--per_device_train_batch_size=2 \
--per_device_eval_batch_size=2 \
--line_by_line \
--learning_rate 5e-5 \
--num_train_epochs 1 \
--save_total_limit 1

2023-12-02 20:18:05.705371: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 20:18:05.705435: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 20:18:05.705483: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-02 20:18:07.722505: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
config.json: 100% 665/665 [00:00<00:00, 2.79MB/s]
vocab.json: 100% 1.04M/1.04M [00:00<00:00, 4.42MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 324MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 5.59MB/s]
/usr/local/lib/python3.10/di

In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/model')
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/model')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
#input_sequence = "Life is"
#input_ids = tokenizer.encode(input_sequence, return_tensor='pt', add_special_tokens=True)

Keyword arguments {'return_tensor': 'pt'} not recognized.


In [15]:
!python3 "/content/drive/MyDrive/generate.py" \
--model_name_or_path '/content/drive/MyDrive/model' \
--length 300 \
--prompt "" \
--stop_token "" \
--k 5 \
--num_return_sequences 10

usage: generate.py [-h] --model_type MODEL_TYPE --model_name_or_path MODEL_NAME_OR_PATH
                   [--prompt PROMPT] [--length LENGTH] [--stop_token STOP_TOKEN]
                   [--temperature TEMPERATURE] [--repetition_penalty REPETITION_PENALTY] [--k K]
                   [--p P] [--seed SEED] [--no_cuda] [--num_return_sequences NUM_RETURN_SEQUENCES]
generate.py: error: the following arguments are required: --model_type
